---

<div align="center">
  <img src="https://raw.githubusercontent.com/devicons/devicon/master/icons/python/python-original.svg" width="80"/>
</div>

<h1 align="center">Programação Orientada a Objetos</h1>

<h3 align="center">PhD. Julles Mitoura</h3>

<div align="center">
  <img src="https://img.shields.io/badge/Python-3776AB?style=for-the-badge&logo=python&logoColor=white"/>
  <img src="https://img.shields.io/badge/Jupyter-F37626?style=for-the-badge&logo=jupyter&logoColor=white"/>
  <img src="https://img.shields.io/badge/POO-4A90E2?style=for-the-badge"/>
</div>

---

## **Aula 07**: Polimorfismo e Abstração
---

Na aula anterior vimos **herança**, onde uma classe filha reaproveita código e estrutura de uma classe mãe.

Agora vamos evoluir para dois conceitos centrais para escrever código **extensível** e **fácil de manter**:

- **Polimorfismo**: *a mesma chamada* (mesma interface) pode ter **comportamentos diferentes**
- **Abstração**: definimos um **contrato** (o que deve existir) e deixamos os detalhes para as classes concretas

### Objetivos da aula

Ao final, você deve conseguir:

- Usar polimorfismo para evitar `if/elif` gigantes por tipo
- Entender a diferença entre *duck typing* e polimorfismo por herança
- Criar contratos com `ABC` + `@abstractmethod` e forçar implementação

### Roteiro

- Parte 1: Polimorfismo (duck typing)
- Parte 2: Polimorfismo com herança (sobrescrita)
- Parte 3: Abstração com `ABC` e `@abstractmethod`
- Exercícios (fixação)

Vamos aos exemplos.

### Parte 1: Polimorfismo (duck typing)

Em Python, **muitas** APIs exploram *duck typing*:

- Se o objeto tem o método/atributo esperado, então ele “serve” para aquela função.
- Isso permite escrever funções que trabalham com **famílias de objetos** sem amarrar em classes específicas.

Exemplo prático: vários formatos de exportação (CSV/JSON/PDF) com a *mesma chamada* (`exportar`).

In [ ]:
class ExportadorCSV:
    def exportar(self, dados):
        cabecalho = ",".join(dados[0].keys())
        linhas = [cabecalho]
        for item in dados:
            linhas.append(",".join(str(v) for v in item.values()))
        return "\n".join(linhas)


class ExportadorJSON:
    def exportar(self, dados):
        import json

        return json.dumps(dados, ensure_ascii=False, indent=2)


def salvar(exportador, dados):
    # Repare: não exigimos herança. Só exigimos a *interface* (método exportar).
    conteudo = exportador.exportar(dados)
    print(conteudo)


dados = [
    {"id": 1, "nome": "Ana"},
    {"id": 2, "nome": "Beto"},
]

salvar(ExportadorCSV(), dados)
print("-" * 30)
salvar(ExportadorJSON(), dados)

---

### Parte 2: Polimorfismo com herança (sobrescrita de método)

Quando existe uma **relação natural “é um”** (*is-a*), herança pode fazer sentido.

A ideia do polimorfismo aqui é: chamamos o mesmo método, mas cada subclasse implementa do seu jeito.

Exemplo prático: **folha de pagamento**. O cálculo do pagamento muda conforme o tipo de contrato.

In [ ]:
class Funcionario:
    def __init__(self, nome, salario_base):
        self.nome = nome
        self.salario_base = salario_base

    def calcular_pagamento(self):
        # “Contrato” informal: subclasses devem sobrescrever.
        raise NotImplementedError


class CLT(Funcionario):
    def __init__(self, nome, salario_base, vale_transporte=0.0):
        super().__init__(nome, salario_base)
        self.vale_transporte = vale_transporte

    def calcular_pagamento(self):
        # Exemplo simplificado: desconto de 6% de VT
        desconto_vt = self.salario_base * 0.06
        return self.salario_base - desconto_vt + self.vale_transporte


class PJ(Funcionario):
    def __init__(self, nome, valor_hora, horas):
        super().__init__(nome, salario_base=0.0)
        self.valor_hora = valor_hora
        self.horas = horas

    def calcular_pagamento(self):
        return self.valor_hora * self.horas


def rodar_folha(funcionarios):
    total = 0.0
    for f in funcionarios:
        pagamento = f.calcular_pagamento()  # mesma chamada, comportamentos diferentes
        print(f"{f.nome}: R$ {pagamento:.2f}")
        total += pagamento
    print(f"TOTAL: R$ {total:.2f}")


rodar_folha(
    [
        CLT("Ana", 3000, vale_transporte=200),
        PJ("Beto", valor_hora=120, horas=20),
    ]
)

---

### Parte 3: Abstração com `ABC` e `@abstractmethod`

A ideia aqui é transformar aquele “contrato informal” (via `NotImplementedError`) em um **contrato obrigatório**.

- Uma classe **abstrata** define *o que* deve existir.
- As classes concretas definem *como* fazer.

Isso evita que alguém “esqueça” de implementar métodos importantes.

Exemplo: diferentes formas de pagamento, mas o sistema só precisa saber que existe `pagar(valor)`.

In [ ]:
from abc import ABC, abstractmethod


class Pagamento(ABC):
    @abstractmethod
    def pagar(self, valor: float) -> str:
        """Efetua o pagamento e devolve uma mensagem/recibo."""
        raise NotImplementedError


class Pix(Pagamento):
    def __init__(self, chave: str):
        self.chave = chave

    def pagar(self, valor: float) -> str:
        if valor <= 0:
            raise ValueError("valor deve ser positivo")
        return f"PIX ({self.chave}): pago R$ {valor:.2f}"


class CartaoCredito(Pagamento):
    def __init__(self, ultimos_4: str, limite: float):
        self.ultimos_4 = ultimos_4
        self.limite = limite

    def pagar(self, valor: float) -> str:
        if valor <= 0:
            raise ValueError("valor deve ser positivo")
        if valor > self.limite:
            raise ValueError("limite insuficiente")
        self.limite -= valor
        return f"Cartão ****{self.ultimos_4}: pago R$ {valor:.2f} (limite restante R$ {self.limite:.2f})"


def processar_pagamentos(pagamentos, valor: float):
    for p in pagamentos:
        print(p.pagar(valor))


# Pagamento()  # descomente para ver: TypeError (classe abstrata não pode ser instanciada)
processar_pagamentos(
    [
        Pix("ana@banco"),
        CartaoCredito("1234", limite=200),
    ],
    50,
)

---

### Extra (opcional): “duck typing” com segurança usando `Protocol`

No Python, o polimorfismo por *duck typing* é muito comum, mas erros aparecem só em **tempo de execução** (ex.: `AttributeError`).

Se você usa *type hints*, dá para manter a flexibilidade do duck typing e ainda ganhar **checagem estática** com `typing.Protocol`.

A ideia: você descreve a *interface estrutural* (métodos esperados), sem exigir herança.

In [ ]:
from typing import Protocol


class Exportador(Protocol):
    def exportar(self, dados) -> str: ...


def salvar_tipado(exportador: Exportador, dados) -> None:
    # Qualquer objeto que tenha exportar(dados)->str é aceito.
    print(exportador.exportar(dados))


# Reaproveita as classes ExportadorCSV e ExportadorJSON definidas anteriormente.
salvar_tipado(ExportadorCSV(), dados)
salvar_tipado(ExportadorJSON(), dados)


class Quebrado:
    pass


# salvar_tipado(Quebrado(), dados)
# Em runtime daria AttributeError; com type checker (mypy/pyright) você já veria o erro antes.

---

## Exercícios (fixação)

1) **Notificações (polimorfismo por duck typing)**

- Crie classes `Email`, `SMS` e `Push` com método `enviar(mensagem: str) -> str`.
- Crie uma função `notificar(canais, mensagem)` que percorre os canais e imprime o retorno de `enviar`.
- Adicione um novo canal `WhatsApp` sem alterar a função `notificar`.

2) **Cálculo de área (abstração com ABC)**

- Crie uma classe abstrata `Forma` com método abstrato `area() -> float`.
- Implemente `Retangulo(largura, altura)` e `Circulo(raio)`.
- Escreva uma função `total_area(formas)` que soma as áreas.

3) **Pagamento (reforçando contrato)**

- Crie `Boleto(Pagamento)` e implemente `pagar(valor)`.
- Garanta que valores inválidos (zero/negativo) gerem `ValueError`.

Dica: o objetivo é você conseguir *adicionar novas classes* sem mexer nas funções que usam a interface.

---

## Resumo

- **Polimorfismo**: escrevemos funções que dependem de uma *interface* (métodos esperados), e cada objeto cumpre essa interface do seu jeito.
- **Abstração**: usamos classes abstratas para declarar um **contrato obrigatório** (o que toda implementação precisa oferecer).

Em termos práticos:

- Polimorfismo ajuda a **evitar** `if tipo == ...` espalhado pelo código.
- Abstração ajuda a **garantir** que novas implementações não “quebrem” a API.

Faça os exercícios e tente **adicionar uma nova classe** em cada cenário sem tocar nas funções (`notificar`, `total_area`, `processar_pagamentos`).

Até a próxima aula!

---